In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import sys
import os
HOME = os.path.expanduser('~')
import datetime
sys.path.append(f'{HOME}/kaggle/data_analysis/model')
from params_HC import params_lgb

sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
import utils
from feature_manage import FeatureManage
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())

#========================================================================
# Global Variable
sys.path.append(f'../py')
from info_home_credit import hcdr_key_cols
key, target, ignore_list = hcdr_key_cols()
#========================================================================

prev_key = 'SK_ID_PREV'

acr = 'AMT_CREDIT'
aan = 'AMT_ANNUITY'
adp = 'AMT_DOWN_PAYMENT'
cpy = 'CNT_PAYMENT'
co_type = 'NAME_CONTRACT_TYPE'
dd = 'DAYS_DECISION'
amt_list = ["f000_app_"+acr, "f000_app_"+aan]

#========================================================================
# Data Load
base = utils.read_df_pkl('../input/base0*')[[key, target]].set_index(key)
manage = FeatureManage(key, target)
manage.set_base(base)
feat_key_list = [['f000_app'], ['f004_app']]
app_train, app_test = manage.feature_matrix(feat_key_list=feat_key_list)
#========================================================================

target = 'CNT_PAYMENT'
ignore_list.remove("TARGET")
ignore_list.append(target)
prev = utils.read_df_pkl('../input/clean_prev*')[[key, dd, acr, aan, cpy]]
train = prev.merge(app_train.drop(amt_list+['TARGET'], axis=1), on=key, how='inner')
test = prev.merge(  app_test.drop(amt_list+['TARGET'], axis=1), on=key, how='inner')

100%|██████████| 3/3 [00:00<00:00,  3.97it/s]


In [14]:
import ml_utils
# from sklearn.model_selection import StratifiedKFold

# df = pd.concat([train, test], axis=0)
# df.drop([dd, 'TARGET'], axis=1, inplace=True)

# train = df[~df[cpy].isnull()]
# test = df[df[cpy].isnull()]
# Y = train[target]

# model_type='lgb'
# learning_rate = 0.05
# early_stopping_rounds = 100
# num_boost_round = 10000
# metric = 'l2'

# seed = 1208
# set_type = 'all'
# n_fold = 5

# folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)
# kfold = folds.split(train, Y)

params = params_lgb()
params['learning_rate'] = learning_rate
params['objective'] = 'regression'
params['metric'] = metric

#========================================================================
# Train & Prediction Start
feim_list = []
score_list = []
oof_pred = np.zeros(len(train))
y_test = np.zeros(len(test))

use_cols = [col for col in train.columns if col not in ignore_list]
x_test = test[use_cols]
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())

for num_fold, (trn_idx, val_idx) in enumerate(kfold):
    x_train, y_train = train[use_cols].iloc[trn_idx, :], Y.iloc[trn_idx]
    x_val, y_val = train[use_cols].iloc[val_idx, :], Y.iloc[val_idx]

    print(f"Fold{num_fold} | Train:{x_train.shape} | Valid:{x_val.shape}")

    score, tmp_oof, tmp_pred, feim, _ = ml_utils.Regressor(
        model_type=model_type
        , x_train=x_train
        , y_train=y_train
        , x_val=x_val
        , y_val=y_val
        , x_test=x_test
        , params=params
        , seed=seed
        , get_score=metric
    )
    feim_list.append(feim.set_index('feature').rename(columns={'importance':f'imp_{num_fold}'}))

    logger.info(f"Fold{num_fold} CV: {score}")
    score_list.append(score)
    oof_pred[val_idx] = tmp_oof
    y_test += tmp_pred

n_feature = len(x_train.columns)
del x_train, y_train, x_val, y_val
gc.collect()

Fold0 | Train:(922398, 694) | Valid:(230601, 694)


MemoryError: 

In [ ]:
feim = pd.concat(feim_list, axis=1)
feim_cols = [col for col in feim.columns if col.count('imp_')]
feim['importance'] = feim[feim_cols].mean(axis=1)
feim.drop(feim_cols, axis=1, inplace=True)
feim.sort_values(by='importance', ascending=False, inplace=True)
feim['rank'] = np.arange(len(feim))+1

cv_score = np.mean(score_list)
logger.info(f'''
#========================================================================
# Model: {model_type}
# CV   : {cv_score}
#========================================================================''')

y_test /= (num_fold+1)

pred_col = 'prediction'
train[pred_col] = oof_pred
test[pred_col] = y_test
train[key] = manage.base_train.index.tolist()
test[key] = manage.base_test.index.tolist()
train[target] = manage.base_train[target].values
test[target] = manage.base_test[target].values
stack_cols = [key, target, pred_col]

df_stack = pd.concat([train[stack_cols], test[stack_cols]], ignore_index=True, axis=0)

#========================================================================
# Saving
feim.to_csv(f'../valid/{start_time[4:12]}_valid_{model_type}_SET-{set_type}_feat{n_feature}_{comment}_CV{str(cv_score)[:7]}_LB.csv', index=True)
utils.to_pkl_gzip(obj=df_stack, path=f'../stack/{start_time[4:12]}_stack_{model_type}_SET-{set_type}_feat{n_feature}_{comment}_CV{str(cv_score)[:7]}_LB')

submit = pd.read_csv('../input/sample_submission.csv').set_index(key)
submit[target] = test[pred_col].values
submit_path = f'../submit/{start_time[4:12]}_submit_{model_type}_SET-{set_type}_feat{n_feature}_{comment}_CV{str(cv_score)[:7]}_LB.csv'
submit.to_csv(submit_path, index=True)

if is_submit:
    utils.submit(file_path=submit_path, comment=comment)
    shutil.move(submit_path, '../log_submit/')

#========================================================================